In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found exi

# Checking data

In [4]:
import os

# === CẤU HÌNH ===
base_dir = "/kaggle/input/data-segment-phase-1/dataset"  # hoặc thư mục chứa 'images/' và 'labels/'

def count_files(folder):
    return len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])

def report_counts(split):
    img_dir = os.path.join(base_dir, "images", split)
    lbl_dir = os.path.join(base_dir, "labels", split)

    img_count = count_files(img_dir) if os.path.exists(img_dir) else 0
    lbl_count = count_files(lbl_dir) if os.path.exists(lbl_dir) else 0

    print(f"📂 {split.capitalize()}:")
    print(f"    Số ảnh   : {img_count}")
    print(f"    Số label : {lbl_count}")
    print(f"    Mismatch : {abs(img_count - lbl_count)}\n")

# === GỌI BÁO CÁO ===
print(" Kiểm tra số lượng ảnh và label:")
for split in ["train", "val"]:
    report_counts(split)


 Kiểm tra số lượng ảnh và label:
📂 Train:
    Số ảnh   : 1618
    Số label : 1618
    Mismatch : 0

📂 Val:
    Số ảnh   : 411
    Số label : 411
    Mismatch : 0



# Setup Yolov8l Segment

In [5]:
# Kiểm tra GPU
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


True
Tesla T4


In [6]:
data_yaml = """
path: /kaggle/input/data-segment-phase-1/dataset
train: images/train
val: images/val
names:
  0: unbet
  1: fanduel
  2: draftkings
  3: bally
  4: gilariver
  5: betrivers
  6: bet365
  7: pointsbet
  8: betmgm
  9: caesars
  10: betparx
  11: betway
  12: fanatics
  13: casino
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml)


In [7]:
from ultralytics import YOLO

# Tải model yolov8l-seg.pt
model = YOLO("Model_yolo_base/yolov8l-seg.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 88.1M/88.1M [00:00<00:00, 236MB/s]


In [9]:

model.train(
    data="data.yaml",           # sử dụng file vừa tạo
    epochs=150,
    imgsz=640,
    optimizer="Adam",
    batch=8,
    device=0,
    name="yolov8l-seg-kaggle",
)

# Nén toàn bộ folder runs vào file zip
!zip -r /kaggle/working/runs_yolo.zip /kaggle/working/runs


Ultralytics 8.3.143 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=Model_yolo_base/yolov8l-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8l-seg-kaggle2, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

train: Scanning /kaggle/input/data-segment-phase-1/dataset/labels/train... 1618 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1618/1618 [00:02<00:00, 704.69it/s]

train: /kaggle/input/data-segment-phase-1/dataset/images/train/Cavaliers_vs_Nets_1305_Thunv10_000054.jpg: 1 duplicate labels removed
train: /kaggle/input/data-segment-phase-1/dataset/images/train/Jayson_Tatum_1605_khaihoan2003_000320.jpg: 1 duplicate labels removed
train: /kaggle/input/data-segment-phase-1/dataset/images/train/The_WILDEST_ENDINGS_24-25_1805_Chaudpb_000095.jpg: 1 duplicate labels removed
train: /kaggle/input/data-segment-phase-1/dataset/images/train/This_Is_Streetball_1405_khaihoan2003_000214.jpg: 1 duplicate labels removed
train: /kaggle/input/data-segment-phase-1/dataset/images/train/This_Is_Streetball_1405_khaihoan2003_000996.jpg: 1 duplicate labels removed
train: /kaggle/input/data-segment-phase-1/dataset/images/train/epicducks_posters_1205_NghiaNT20_000077.jpg: 2 duplicate labels removed
train: /kaggle/input/data-segment-phase-1/dataset/images/train/epicducks_posters_1205_NghiaNT20_001899.jpg: 1 duplicate labels removed
train: /kaggle/input/data-segment-phase-1/dat

val: Fast image access ✅ (ping: 1.8±3.2 ms, read: 311.0±140.0 MB/s, size: 368.1 KB)


val: Scanning /kaggle/input/data-segment-phase-1/dataset/labels/val... 411 images, 0 backgrounds, 0 corrupt: 100%|██████████| 411/411 [00:00<00:00, 574.36it/s]

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:10<00:00,  2.56it/s]


                   all        411       3957      0.863      0.791       0.83      0.702      0.841      0.772        0.8      0.505
                 unbet        388       3340      0.945      0.777      0.908      0.707      0.902      0.742      0.839      0.445
               fanduel        161        277      0.975      0.859      0.945      0.779      0.926      0.816        0.9       0.48
            draftkings         49         59      0.981      0.875      0.941      0.845      0.962      0.859      0.925      0.698
                 bally         52         56      0.925      0.893      0.929      0.713      0.795      0.768      0.682      0.273
             gilariver         29         33       0.96       0.97      0.967      0.897       0.96       0.97      0.967        0.7
             betrivers         13         17      0.877      0.838      0.892      0.717      0.877      0.838      0.892      0.507
                bet365         23         43      0.955      0.628   

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 12.2ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/segment/yolov8l-seg-kaggle2
updating: kaggle/working/runs/ (stored 0%)
updating: kaggle/working/runs/segment/ (stored 0%)
updating: kaggle/working/runs/segment/yolov8l-seg-kaggle/ (stored 0%)
updating: kaggle/working/runs/segment/yolov8l-seg-kaggle/MaskPR_curve.png (deflated 13%)
updating: kaggle/working/runs/segment/yolov8l-seg-kaggle/labels_correlogram.jpg (deflated 37%)
updating: kaggle/working/runs/segment/yolov8l-seg-kaggle/MaskF1_curve.png (deflated 13%)
updating: kaggle/working/runs/segment/yolov8l-seg-kaggle/results.csv (deflated 52%)
updating: kaggle/working/runs/segment/yolov8l-seg-kaggle/train_batch2.jpg (deflated 5%)
updating: kaggle/working/runs/segment/yolov8l-seg-kaggle/MaskR_curve.png (deflated 12%)
updating: kaggle/working/runs/segment/yolov8l-seg-kaggle/val_batch1_labels.jpg (deflated 4%)
updating: kaggle/working/runs/segment/yolov8l-seg-kaggle/val_batch0_labels.jp